# LOL Models

## Data Export
```
# dump player LOL data
dumpdata.sc --seasons 2021 2022 --stats "*" --progress lol_hist_2014-2022.scored.db --no_teams --target_calc_stats dk_performance_score fd_performance_score --hist_recent_games 3 --hist_recent_mode ma --extra_stats "*" -f lol_train_player.csv

# dumpe team LOL data
dumpdata.sc --seasons 2020 2021 2022 --stats "*" --progress lol_hist_2014-2022.scored.db --no_players --target_calc_stats dk_performance_score fd_performance_score --hist_recent_games 3 --hist_recent_mode ma --extra_stats "*" -f lol_train_team.csv
```

In [ ]:
import pandas as pd

RANDOM_SEED = 1
TRAINING_TIME = 1800

lol_player_df = pd.read_csv("/fantasy/lol_train_player.csv")

In [ ]:
import sklearn.model_selection

lol_player_df.pos = lol_player_df.pos.astype(str)
training_cols = [
    col
    for col in lol_player_df
    if col == "pos" or col.startswith("extra") or ":recent" in col or ":std" in col
]
X = pd.get_dummies(lol_player_df[training_cols], columns=["pos"])
y = lol_player_df["calc:dk_performance_score"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, random_state=RANDOM_SEED
)

In [ ]:
import autosklearn.regression
import sklearn.model_selection
import sklearn.metrics
from pprint import pprint

sk_automl = autosklearn.regression.AutoSklearnRegressor(
    seed=RANDOM_SEED, time_left_for_this_task=TRAINING_TIME, memory_limit=6144
)
sk_automl.fit(X_train, y_train)
print(sk_automl.leaderboard())
pprint(sk_automl.show_models(), indent=4)
y_hat = sk_automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, y_hat))


In [ ]:
from tpot import TPOTRegressor

tpot_automl = TPOTRegressor(
    random_state=RANDOM_SEED,
    max_time_mins=TRAINING_TIME / 60,
    verbosity=2,
)
tpot_automl.fit(X_train, y_train)
pprint(tpot_automl.fitted_pipeline_)
y_hat = tpot_automl.predict(X_test)
print("Test R2 score:", sklearn.metrics.r2_score(y_test, y_hat))
